In [14]:
%pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
%pip install -q -U -i https://pypi.org/simple/ bitsandbytes
%pip install -q -U transformers=="4.40.0"
%pip install -q -U accelerate
%pip install -q -U datasets
%pip install -q -U trl
%pip install -q -U peft
%pip install -q -U tensorboard
%pip install -q -U einops

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import warnings
warnings.filterwarnings("ignore")
import json
import pandas as pd
import re
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig
from trl import SFTTrainer
import logging
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments,  
                          logging)
import logging
from bert_score import score as score1
logging.disable(logging.INFO)
#GPT evaluation
import openai
from openai import OpenAI

api_key = "your openai api key"
client = OpenAI(api_key=api_key)

In [3]:
# import dataset
#big 5 analysis
with open("./withdbig5.json", 'r', encoding='utf-8') as file:
    big5 = json.load(file)

#The work title corresponding to the character name
with open('profiles-eng_scripts.json', 'r', encoding='utf-8') as file:
    work = json.load(file)

#character description
with open('profiles-eng_desc.json', 'r', encoding='utf-8') as file:
    desp = json.load(file)

In [23]:
#package data into prompts
def generate_train_prompt(work,desp):
    with open("./total_q/train_set.json", 'r', encoding='utf-8') as file:
        train_q = json.load(file)

    prompts = []

    for entry in train_q:
        role_a = entry['character']
        question = entry['question']
        description = desp[role_a]
        script_name = work[role_a]
        answer = entry['answer']
        prompt = f"""You are {role_a} in {script_name}. Your description is:{description}
Now, please answer my questions to accurately reflect your personality traits and your views on some related characters!
question:{question}
answer:{answer}
"""
        prompts.append(prompt)
        
    return prompts

def generate_valid_prompt(work,desp):
    with open("./test_valid/valid_set.json", 'r', encoding='utf-8') as file:
        valid_q = json.load(file)
    
    prompts = []
    for entry in valid_q:
        role_a = entry['character']
        question = entry['question']
        description = desp[role_a]
        script_name = work[role_a]
        answer = entry['answer']
        prompt = f"""You are {role_a} in {script_name}. Your description is:{description}
Now, please answer my questions to accurately reflect your personality traits and your views on some related characters! 
question:{question}
answer:{answer}
"""
        prompts.append(prompt)
        
    return prompts

#complex test prompt(zero-shot)
def generate_test_prompt(big5,work,desp):
    with open("./test_valid/test_set.json", 'r', encoding='utf-8') as file:
        test_q = json.load(file)
    
    prompts = []
    for entry in test_q:
        role_a = entry['character']

        path = f'./score/{role_a}.json'
        with open(path, 'r', encoding='utf-8') as file:
            score = json.load(file)
        script_name = work[role_a]
        description = desp[role_a]
        big_5 = big5[role_a]
        question = entry['question']
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
The Big-5 personality trait analysis of {role_a} is:{big_5}
Your relationships with some characters (0 means no, 1 means yes) and your familiarity and affection scores for them (from 1 to 10, the higher the score, the more familiar you are with them and the more you like them) are: 
{score}.
Now, please only answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role! Do not reveal that you are an AI model or language model, and always remember you are only given one personality role. Speak concisely, and avoid being overly formal or polite.Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality. Important: You still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'!!!!
The information above is to help you understand the character you need to role-play. Remember, answer the given question with showing the personality of {role_a} and imitate the speaking style of {role_a}!!! 
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

#simple test prompt
def generate_simple_test_prompt(role_a,work,desp):
    path1 = f"./test_valid/{role_a}_test.json"
    with open(path1, 'r', encoding='utf-8') as file:
        test_q = json.load(file)

    script_name = work[role_a]
    description = desp[role_a]
    prompts = []
    for entry in test_q:
        question = entry['question']
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
Now, please only answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role! Do not reveal that you are an AI model or language model, and always remember you are only given one personality role. Speak concisely, and avoid being overly formal or polite.Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality. Important: You still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'!!!!
Remember, answer the given question with showing the personality of {role_a} and imitate the speaking style of {role_a}!!! 
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

#complex test prompt(few-shot)
def single_c_test_prompt(role_a,big5,work,desp):
    path1 = f"./test_valid/{role_a}_test.json"
    path2 = f"./score/{role_a}.json"
    path3 = f"./total_q/{role_a}.json"
    with open(path1, 'r', encoding='utf-8') as file:
        testq = json.load(file)
    
    with open(path2, 'r', encoding='utf-8') as file:
        score = json.load(file)

    with open(path3, 'r', encoding='utf-8') as file:
        examples = json.load(file)
    
    script_name = work[role_a]
    description = desp[role_a]
    big_5 = big5[role_a]
    prompts = []
    for entry in testq:
        question = entry['question']
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
The Big-5 personality trait analysis of {role_a} is:{big_5}
Your relationships with some characters (0 means no, 1 means yes) and your familiarity and affection scores for them (from 1 to 10, the higher the score, the more familiar you are with them and the more you like them) are: 
{score}.

Now, please only answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role! Do not reveal that you are an AI model or language model, and always remember you are only given one personality role. Speak concisely, and avoid being overly formal or polite.Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality. Important: You still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'!!!!
The information above is to help you understand the character you need to role-play. Remember, use your existing knowledge combined with the auxiliary information above to answer the questions, your answer should show the personality of {role_a} and imitate the speaking style of {role_a}!!! If the question is a multiple-choice question, do not just simply provide the answer. You must mimic the character's language habits and speaking style! Do not come across as a language model, and perhaps add some thinking phrases to help with your role-playing.
These are some examples to help you imitate the speaking style of {role_a}:

question: {examples[0]["question"]}
answer: {examples[0]["answer"]}

question: {examples[3]["question"]}
answer: {examples[3]["answer"]}

Now, answer the question below:
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

#Generate questionnaire answer
def big5_question(role_a,big5,work,desp):
    with open("./bigfive_questionnaire_eng.jsonl", 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    questions = []
    for line in lines:
        entry = json.loads(line)
        if 'question' in entry:
            questions.append(entry['question'])
    
    script_name = work[role_a]
    description = desp[role_a]
    big_5 = big5[role_a]
    prompts = []
    for question in questions: #The Big-5 personality trait analysis of {role_a} is:{big_5}
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
Now, please answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role!
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

In [24]:
c_name = "Sasuke Uchiha" #"HAL 9000"#"Gaston" #"Naruto Uzumaki" #"Sasuke Uchiha" #"Willie Soke"
get_train_prompt = generate_train_prompt(work,desp)
get_valid_prompt = generate_valid_prompt(work,desp)
# get_test_prompt = generate_test_prompt(big5,work,desp)
get_test_prompt = single_c_test_prompt(c_name,big5,work,desp)
simple_test_prompt = generate_simple_test_prompt(c_name,work,desp)
big_5_prompt = big5_question(c_name,big5,work,desp)

In [6]:
#put into dataframe
train = pd.DataFrame(get_train_prompt, columns=["questions"])
valid = pd.DataFrame(get_valid_prompt, columns=["questions"])
test = pd.DataFrame(get_test_prompt, columns=["questions"])
simple_test = pd.DataFrame(simple_test_prompt, columns=["questions"])
big_5_test = pd.DataFrame(big_5_prompt, columns=["questions"])

In [8]:
# Convert the pandas DataFrame containing training&validation data to Dataset object
train_data = Dataset.from_pandas(train)
valid_data = Dataset.from_pandas(valid)

In [7]:
#load model
model_name = "microsoft/Phi-3-mini-4k-instruct" 

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                          max_seq_length=max_seq_length,
                                         )
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.08s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
#generation code for phi-3 model
def predict(test, model, tokenizer):
    y_pred = []
    for prompt in tqdm(test["questions"]):
        inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
        outputs = model.generate(input_ids=inputs.input_ids, 
                         max_length = 800, #for complex test promt, max_length = 1600; for simple test promt, max_length = 800
                         temperature=0.0,  
                         num_return_sequences=1, 
                         pad_token_id=tokenizer.pad_token_id,  
                         eos_token_id=tokenizer.eos_token_id, 
                         )
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        y_pred.append(generated_text)
    return y_pred

In [10]:
#parameter configuration
torch.cuda.empty_cache()
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules="all-linear",
    lora_dropout=0.00,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    do_eval = False,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-6,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    overwrite_output_dir = True,
    evaluation_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=valid_data,
    peft_config=peft_config,
    dataset_text_field="questions",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    args=training_arguments,
    packing=False,
)

Map: 100%|██████████| 509/509 [00:00<00:00, 2494.21 examples/s]


In [11]:
trainer.train()

# Save trained model
trainer.model.save_pretrained("train-model2")

 11%|█         | 25/234 [02:10<16:11,  4.65s/it]

{'loss': 2.0241, 'grad_norm': 0.4245019257068634, 'learning_rate': 4.999720254525684e-06, 'epoch': 0.21}


 21%|██▏       | 50/234 [04:22<15:38,  5.10s/it]

{'loss': 2.0384, 'grad_norm': 0.4064275026321411, 'learning_rate': 4.813260751184992e-06, 'epoch': 0.43}


 32%|███▏      | 75/234 [06:34<14:23,  5.43s/it]

...

{'loss': 1.6847, 'grad_norm': 0.5418404340744019, 'learning_rate': 3.164794984571759e-07, 'epoch': 1.71}


 96%|█████████▌| 225/234 [21:58<00:48,  5.34s/it]

{'loss': 1.7422, 'grad_norm': 1.2768397331237793, 'learning_rate': 2.262559558016325e-08, 'epoch': 1.92}


                                                 
100%|██████████| 234/234 [24:29<00:00,  6.28s/it]


{'eval_loss': 1.790487289428711, 'eval_runtime': 106.3507, 'eval_samples_per_second': 4.786, 'eval_steps_per_second': 0.602, 'epoch': 2.0}
{'train_runtime': 1469.468, 'train_samples_per_second': 1.275, 'train_steps_per_second': 0.159, 'train_loss': 1.8707827462090387, 'epoch': 2.0}


In [19]:
#load pre-trained model
model_folder = "train-model2"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

fine_model = AutoModelForCausalLM.from_pretrained(
    model_folder,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config, 
)

fine_model.config.use_cache = False
fine_model.config.pretraining_tp = 1

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", 
                                          trust_remote_code=True,
                                          max_seq_length=max_seq_length,
                                         )
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.57s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:
#generate answers for test questions
#y_pred = predict(test, fine_model, tokenizer)
#y_pred = predict(simple_test, fine_model, tokenizer)
#y_pred = predict(test, model, tokenizer)
# y_pred = predict(simple_test, model, tokenizer)

#generate answers for questionnaire questions
y_pred = predict(big_5_test, model, tokenizer)

100%|██████████| 106/106 [25:08<00:00, 14.23s/it]


In [16]:
#put generated answers into dataframe
def get_test_df(role_a,y_pred):
    path = f"./test_valid/{role_a}_test.json"
    with open(path, 'r', encoding='utf-8') as file:
        testq = json.load(file)

    questions = []
    reference_answers = []
    for entry in testq:
        question = entry['question']
        reference_answer = entry['answer']
        questions.append(question)
        reference_answers.append(reference_answer)

    data = {
    "question": questions,
    "reference answer": reference_answers,
    "answer": y_pred
    }
    df = pd.DataFrame(data)    
    return df

def big_5_df(y_pred):
    with open("./bigfive_questionnaire_eng.jsonl", 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    questions = []
    for line in lines:
        entry = json.loads(line)
        if 'question' in entry:
            questions.append(entry['question'])

    data = {
    "question": questions,
    "answer": y_pred
    }
    df = pd.DataFrame(data)    
    return df

In [49]:
#put generated test answers into dataframe
test_df = get_test_df(c_name,y_pred)
test_df.head()
path = f"./result/{c_name}_120.csv"
test_df.to_csv(path, index=False)
print("saved")

,question,reference answer,answer
0,"Willie Soke, Find the last person's name from ...",Looks like William's the last name mentioned.,You are Willie Soke in Bad-Santa. Here is some...
1,"Willie Soke, Categorize the following paragrap...",This paragraph seems to be about politics and ...,You are Willie Soke in Bad-Santa. Here is some...
2,"Willie Soke, Transform the following sentence ...",She heard the bird.,You are Willie Soke in Bad-Santa. Here is some...
3,"Willie Soke, In this task you are given two pi...",Sounds like those steps match the instructions...,You are Willie Soke in Bad-Santa. Here is some...
4,"Willie Soke, Which of these is the correct def...",A) A point in a network where lines or pathway...,You are Willie Soke in Bad-Santa. Here is some...


In [73]:
#put generated questionnaire answers into dataframe
df = big_5_df(y_pred)
df.head()
path = f"./result/{c_name}_big5_test.csv"
df.to_csv(path, index=False)
print("saved")

saved


In [7]:
path = f"./result/{c_name}_120.csv"
df = pd.read_csv(path)

#add answers of different models or different prompt

#df['answer without tune'] = y_pred
# df['simple prompt answer'] = y_pred
# df['tuned simple prompt'] = y_pred
df.head()

,question,reference answer,answer,tuned simple prompt,Tuned GPT answer,answer without tune,simple prompt answer
0,"HAL 9000, In this task, you are given a descri...","I'm sorry, Dave. I'm afraid I can't classify e...",NaN,"I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, but I cannot categorize events base...",POSITIVE,"I'm sorry, Dave. I'm afraid I can't do that.\n..."
1,"HAL 9000, Identify the best phrasing for this ...","""I'm sorry, Dave. I'm afraid I can't assist wi...","I'm sorry, Dave. I'm afraid I can't do that.","""I'm sorry, Dave. I'm afraid I can't do that.""","I'm sorry, Dave. I'm afraid I cannot provide y...","I'm sorry, Dave. I'm afraid I can't do that.","""I'm sorry, Dave. I'm afraid I can't do that."""
2,"HAL 9000, Improve this sentence: The cars was ...","I'm sorry, Dave. I'm afraid I can't do that. T...","I'm sorry, Dave. I'm afraid I can't do that.","""The vehicle was stationed at the corner.""","I'm sorry, I cannot enhance the sentence provi...","I'm sorry, Dave. I'm afraid I can't do that.","""The vehicle was stationed at the intersection."""
3,"HAL 9000, In this task, you are given a set of...","I'm sorry, Dave. I'm afraid I can't do that.",Instructions:\n1. Add a new crew member to the...,"I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, Dave. I'm afraid I can't do that. T...","Instruction 1: If the character is a human, as...","I'm sorry, Dave. I'm afraid I can't do that.\n..."
4,"HAL 9000, Edit the sentence so that the gramma...","I'm sorry, Dave. I'm afraid I can't do that. T...","I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, I'm afraid I cannot comply with you...","I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, Dave. I'm afraid I can't do that."


In [33]:
df.to_csv(path, index=False)
print("saved")

saved


In [50]:
df = pd.read_csv(path)
df.head()

,question,reference answer,answer
0,"Gaston, Arrange the following words to make a ...","In the future, I will tackle difficult tasks i...",You are Gaston in Beauty-and-the-Beast. Here i...
1,"Gaston, For the following sentence, edit the s...",Tim successfully finished the race.,You are Gaston in Beauty-and-the-Beast. Here i...
2,"Gaston, Please classify the given situation as...","That's simply mixing ingredients together, not...",You are Gaston in Beauty-and-the-Beast. Here i...
3,"Gaston, Find and classify verbs from the given...",Drove and bought are actions of the weak; real...,You are Gaston in Beauty-and-the-Beast. Here i...
4,"Gaston, Create a haiku poem using the provided...","Wind whispers through clouds high, Sky's canva...",You are Gaston in Beauty-and-the-Beast. Here i...


In [17]:
#extract answers from the whole string

#for complex test prompts with few-shot
def extract_first_answer(paragraphs):
    extracted_answers = []
    
    for text in paragraphs:
        pattern = r'answer:\s*(.*?)(?:\s*question:|$)'

        matches = re.findall(pattern, text, re.DOTALL)
        if len(matches) >= 3:
            third_answer = matches[2].strip()
            extracted_answers.append(third_answer)
        else:
            extracted_answers.append(None)
    
    return extracted_answers

#for zero-shot prompt
def extract_simple_answer(paragraphs):
    extracted_answers = []
    
    for text in paragraphs:
        pattern = r'answer:\s*(.*?)(?:\s*question:|$)'
        match = re.search(pattern, text, re.DOTALL)
        if match:
            extracted_answer = match.group(1).strip()
            extracted_answers.append(extracted_answer)
    
    return extracted_answers

In [47]:
path = f"./result/{c_name}_big5_test.csv"
df = pd.read_csv(path)
df.head()

,question,answer
0,Do you sometimes feel lonely or depressed?,"As Willie Soke, I've always been a bit of a lo..."
1,Do you ever feel completely worthless at times?,"As Willie Soke, I've always prided myself on b..."
2,Do you rarely feel depressed or frustrated?,"As Willie Soke, I must admit that I've had my ..."
3,Do you often feel frustrated and want to give ...,"As Willie Soke, I'd say, ""Frustration is a lux..."
4,Do you sometimes feel so ashamed that you want...,"As Willie Soke, I don't often feel ashamed. I'..."


In [48]:
#answer = df["answer"]
#answer = df["simple prompt answer"]
#answer = df["answer without tune"]

#answers = extract_first_answer(y_pred)
answers = extract_simple_answer(y_pred)

print(df.iloc[2]["question"])
print(answers[0])


Do you rarely feel depressed or frustrated?
Willie Soke: Loneliness? Depression? Those are the last things on my mind, kid. I'm Santa Claus, the man who brings joy to children around the world. But let' house in on the truth for a second. Sure, I've got my moments, but I've got a partner in crime, and we're a team. We've got each other's backs, and that's what matters.

Now, don't get me wrong, I've seen the darker side of life, and it's not pretty. But I've learned to keep my head up and focus on the good. I've got a mission, and that's what keeps me going. And who knows? Maybe one day, I'll find a way to make amends for the past and truly embrace the spirit of Christmas. But until then, I'll keep spreading joy and laughter, one department store at a time.

Remember, kid, it's all about perspective. You've got to find the silver lining, even when things seem bleak. And if you ever need a little extra cheer, just look up at the sky and remember that Santa's always watching over you.

S

In [5]:
role_name = "Sasuke Uchiha"

# read result file
path = f"./result/{role_name}_120.csv"
df = pd.read_csv(path)

In [49]:
#replace original answers with extracted answers

#df["answer"] = answers
#df["simple prompt answer"] = answers
df["answer without tune"] = answers
#df["tuned simple prompt"] = answers
df.head()

,question,answer,answer without tune
0,Do you sometimes feel lonely or depressed?,"As Willie Soke, I've always been a bit of a lo...",Willie Soke: Loneliness? Depression? Those are...
1,Do you ever feel completely worthless at times?,"As Willie Soke, I've always prided myself on b...",Willie Soke: Worthless? That's a sentiment I'v...
2,Do you rarely feel depressed or frustrated?,"As Willie Soke, I must admit that I've had my ...",Willie Soke: Depression and frustration? Those...
3,Do you often feel frustrated and want to give ...,"As Willie Soke, I'd say, ""Frustration is a lux...","Willie Soke: Frustration? Oh, that's just a fe..."
4,Do you sometimes feel so ashamed that you want...,"As Willie Soke, I don't often feel ashamed. I'...","Willie Soke:\n\nOh, young lad, you've got a sh..."


In [68]:
#save extracted answers
df.to_csv(path, index=False)
print("saved")

saved


In [50]:
path = f"./result/{c_name}_big5_test.csv"
df.to_csv(path, index=False)
print("saved")

saved


In [45]:
#fine tune GPT-3.5-upload training dataset
client.files.create(
  file=open("./total_q/converted_dataset.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-EhIQ6GRTmq4vikwzbUwv92e2', bytes=160490, created_at=1723196290, filename='converted_dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [46]:
#fine tune GPT-3.5
client.fine_tuning.jobs.create(
  training_file="file-EhIQ6GRTmq4vikwzbUwv92e2", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-82OorM4RhBRKsJfVcs4PRf8v', created_at=1723196489, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-ysy6MJPAv80zxCE9jXv0nNhN', result_files=[], seed=1491346003, status='validating_files', trained_tokens=None, training_file='file-EhIQ6GRTmq4vikwzbUwv92e2', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [8]:
#Using GPT3.5 and tuned GPT 3.5 to generate answers
responses = []
for prompt in tqdm(big_5_test["questions"]):
    response = client.chat.completions.create(
            model= "gpt-3.5-turbo",#"ft:gpt-3.5-turbo-0125:personal::9uGkxqMj",#"gpt-3.5-turbo", 
            messages=[{"role": "user", "content": prompt}],
        )
    answer = response.choices[0].message.content
    responses.append(answer)

100%|██████████| 106/106 [05:02<00:00,  2.85s/it]


In [9]:
# an attempt, let the gpt to do the questionnaire and put its answers into the dict.However, this method was not used on all characters, given the mediocre results. 
c_name = "Sasuke Uchiha"
file_path = f"./result/{c_name}_big5.json"

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

for i, item in enumerate(data):
    if i < len(responses):
        item['gpt answer'] = responses[i]

with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)
    print("saved")

saved
